# Improved Human Pose Classification using GCN

This notebook implements the enhanced training pipeline for classifying human poses using Graph Convolutional Networks (GCN) with the following improvements:

1. Deeper GCN architecture with residual connections
2. Data augmentation for better generalization
3. Class weighting for balanced training
4. Early stopping to prevent overfitting
5. Learning rate scheduling for better convergence
6. Regular checkpointing during training

In [ ]:
# Import required libraries
import torch
from torch_geometric.loader import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import os

# Import our modules
import sys
sys.path.append('src')
from data_processing import load_dataset, PoseAugmentation
from gcn_model import PoseGCN, DeepPoseGCN

print("All libraries imported successfully!")

In [10]:
# Add early stopping utility
class EarlyStopping:
    """Early stops the training if validation score doesn't improve after a given patience."""
    def __init__(self, patience=50, verbose=True, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): How long to wait after last improvement.
            verbose (bool): If True, prints a message for each validation improvement.
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
            path (str): Path for the checkpoint to be saved to.
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_score_max = 0
        self.delta = delta
        self.path = path

    def __call__(self, val_score, model):
        # Higher score is better (e.g., F1)
        score = val_score

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_score, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_score, model)
            self.counter = 0

    def save_checkpoint(self, val_score, model):
        '''Saves model when validation score improves.'''
        if self.verbose:
            print(f'Validation score improved ({self.val_score_max:.6f} --> {val_score:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_score_max = val_score

## Check Data Paths

In [ ]:
# Define and verify data paths
data_dir = os.path.join(os.getcwd(), 'data_v2')
annotation_dir = os.path.join(os.getcwd(), 'annotations_v2')

# Check if directories exist
print(f"Checking data directory: {data_dir}")
print(f"Exists: {os.path.exists(data_dir)}")

print(f"\nChecking annotation directory: {annotation_dir}")
print(f"Exists: {os.path.exists(annotation_dir)}")

# Check annotation files
annotation_files = [
    't5-sherul-300-195-correct.json',
    'lumbar-K-1.1-160.json'
]

print("\nChecking annotation files:")
for file in annotation_files:
    file_path = os.path.join(annotation_dir, file)
    print(f"{file}: {'Exists' if os.path.exists(file_path) else 'Missing'}")

## Define Training and Evaluation Functions

In [12]:
def train(model, train_loader, optimizer, device, class_weights=None):
    model.train()
    total_loss = 0
    
    for data in tqdm(train_loader, desc='Training', leave=False):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Use weighted loss if class weights are provided
        if class_weights is not None:
            loss = torch.nn.functional.nll_loss(output, data.y, weight=class_weights)
        else:
            loss = torch.nn.functional.nll_loss(output, data.y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(train_loader)

In [13]:
def evaluate(model, loader, device):
    model.eval()
    predictions = []
    labels = []
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            pred = output.max(dim=1)[1]
            
            predictions.extend(pred.cpu().numpy())
            labels.extend(data.y.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    
    return accuracy, precision, recall, f1

In [14]:
def plot_metrics(train_losses, val_metrics, save_path='models/training_metrics.png'):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot training loss
    ax1.plot(train_losses, 'b-', label='Training Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.grid(True)
    ax1.legend()
    
    # Plot validation metrics
    epochs = range(len(val_metrics['accuracy']))
    ax2.plot(epochs, val_metrics['accuracy'], 'g-', label='Accuracy')
    ax2.plot(epochs, val_metrics['precision'], 'r-', label='Precision')
    ax2.plot(epochs, val_metrics['recall'], 'b-', label='Recall')
    ax2.plot(epochs, val_metrics['f1'], 'y-', label='F1-Score')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Score')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(save_path)
    plt.show()
    print(f"Saved training plot to {save_path}")

In [15]:
def train_and_evaluate(model, train_loader, val_loader, optimizer, num_epochs=500, device='cpu', 
                       class_weights=None, early_stopping_patience=50, checkpoint_dir='models'):
    best_f1 = 0
    train_losses = []
    val_metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    learning_rates = [optimizer.param_groups[0]['lr']]  # Track learning rates
    
    # Initialize early stopping
    model_name = model.__class__.__name__
    early_stopping = EarlyStopping(
        patience=early_stopping_patience, 
        verbose=True, 
        path=f'{checkpoint_dir}/model_{model_name}_early_stop.pth'
    )
    
    # Initialize learning rate scheduler
    scheduler = ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=30, verbose=True
    )
    
    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    
    epoch_checkpoints = [50, 100, 200, 300, 400]  # Save at these specific epochs
    
    for epoch in range(num_epochs):
        # Train
        train_loss = train(model, train_loader, optimizer, device, class_weights)
        train_losses.append(train_loss)
        
        # Evaluate
        val_acc, val_prec, val_rec, val_f1 = evaluate(model, val_loader, device)
        
        # Store metrics
        val_metrics['accuracy'].append(val_acc)
        val_metrics['precision'].append(val_prec)
        val_metrics['recall'].append(val_rec)
        val_metrics['f1'].append(val_f1)
        
        # Print progress
        print(f'Epoch {epoch+1:03d}:')
        print(f'Train Loss: {train_loss:.4f}')
        print(f'Val Accuracy: {val_acc:.4f}, Precision: {val_prec:.4f}, '
              f'Recall: {val_rec:.4f}, F1: {val_f1:.4f}')
        
        # Update learning rate based on validation F1 score
        scheduler.step(val_f1)
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)  # Track learning rate changes
        print(f'Current learning rate: {current_lr:.6f}')
        
        # Save best model whenever a new best F1 score is achieved
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_metrics': val_metrics,
                'train_losses': train_losses,
                'learning_rate': current_lr,
                'learning_rates': learning_rates
            }, f'{checkpoint_dir}/model_{model_name}_best.pth')
            print(f"Saved best model with F1: {val_f1:.4f}")
        
        # Save checkpoint at specific epoch milestones
        if (epoch + 1) in epoch_checkpoints:
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_metrics': val_metrics,
                'train_losses': train_losses,
                'learning_rate': current_lr,
                'learning_rates': learning_rates
            }, f'{checkpoint_dir}/model_{model_name}_epoch_{epoch+1}.pth')
            print(f"Saved checkpoint at epoch {epoch+1}")
            
        # Early stopping - Stops training if there's no improvement
        early_stopping(val_f1, model)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
            
    return train_losses, val_metrics, learning_rates

## Setup Data With Augmentation

In [ ]:
try:
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create augmentation transform
    augmentation = PoseAugmentation(
        noise_level=0.02,  # 2% noise relative to the normalized keypoints
        drop_edge_prob=0.1,  # 10% probability to drop an edge
        invisible_prob=0.1,  # 10% probability to mark a keypoint as invisible
        p=0.5  # 50% probability to apply augmentation to a sample
    )
    
    # Load datasets with augmentation
    train_dataset, val_dataset = load_dataset(
        root_dir=data_dir,
        annotation_dir=annotation_dir
    )
    
    # Apply augmentation to training dataset
    train_dataset.transform = augmentation
    
    print(f"Dataset loaded: {len(train_dataset)} training samples, {len(val_dataset)} validation samples")
    
    # Compute class weights for balanced training
    train_labels = [data.y.item() for data in train_dataset]
    class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
    class_weights = torch.FloatTensor(class_weights).to(device)
    print(f"Class weights: {class_weights}")
    
    # Create data loaders
    batch_size = 32
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Create models directory and its subdirectories
    os.makedirs('models/original', exist_ok=True)
    os.makedirs('models/deep', exist_ok=True)
    
    print("Setup complete!")
    
except Exception as e:
    print(f"Error during setup: {str(e)}")
    raise

## Train Original PoseGCN Model (Baseline)

In [ ]:
# Initialize and train the original model for comparison
original_model = PoseGCN(num_node_features=2).to(device)
original_optimizer = optim.Adam(original_model.parameters(), lr=0.001)

print("\n=== Training Original PoseGCN ===")
original_losses, original_metrics, original_lrs = train_and_evaluate(
    original_model, train_loader, val_loader, original_optimizer, 
    num_epochs=500, device=device, class_weights=class_weights,
    early_stopping_patience=50, checkpoint_dir='models/original'
)

# Plot metrics for original model
plot_metrics(original_losses, original_metrics, save_path='models/original/original_model_metrics.png')

## Train Deep PoseGCN Model (Enhanced)

In [ ]:
# Initialize and train the deep model
deep_model = DeepPoseGCN(num_node_features=2).to(device)
deep_optimizer = optim.Adam(deep_model.parameters(), lr=0.001, weight_decay=1e-4)  # Added weight decay for regularization

print("\n=== Training DeepPoseGCN ===")
deep_losses, deep_metrics, deep_lrs = train_and_evaluate(
    deep_model, train_loader, val_loader, deep_optimizer, 
    num_epochs=500, device=device, class_weights=class_weights,
    early_stopping_patience=50, checkpoint_dir='models/deep'
)

# Plot metrics for deep model
plot_metrics(deep_losses, deep_metrics, save_path='models/deep/deep_model_metrics.png')

## Compare Model Performance

In [ ]:
# Compare the best performance of both models
print("\n=== Model Comparison ===")
print(f"Original PoseGCN - Best F1: {max(original_metrics['f1']):.4f}")
print(f"DeepPoseGCN - Best F1: {max(deep_metrics['f1']):.4f}")

# Add comparison of best precision, recall, and accuracy values
print("\n=== Best Precision, Recall, Accuracy Comparison ===")
print(f"Original PoseGCN - Best Precision: {max(original_metrics['precision']):.4f}")
print(f"DeepPoseGCN - Best Precision: {max(deep_metrics['precision']):.4f}")
print("\nOriginal PoseGCN - Best Recall: {:.4f}".format(max(original_metrics['recall'])))
print("DeepPoseGCN - Best Recall: {:.4f}".format(max(deep_metrics['recall'])))
print("\nOriginal PoseGCN - Best Accuracy: {:.4f}".format(max(original_metrics['accuracy'])))
print("DeepPoseGCN - Best Accuracy: {:.4f}".format(max(deep_metrics['accuracy'])))

# Find the epochs with the best metrics for both models
original_best_f1_epoch = np.argmax(original_metrics['f1'])
deep_best_f1_epoch = np.argmax(deep_metrics['f1'])
print("\n=== Best Model Details (Based on F1 Score) ===")
print(f"Original PoseGCN - Best epoch: {original_best_f1_epoch+1}")
print(f"  F1: {original_metrics['f1'][original_best_f1_epoch]:.4f}")
print(f"  Precision: {original_metrics['precision'][original_best_f1_epoch]:.4f}")
print(f"  Recall: {original_metrics['recall'][original_best_f1_epoch]:.4f}")
print(f"  Accuracy: {original_metrics['accuracy'][original_best_f1_epoch]:.4f}")
print(f"\nDeepPoseGCN - Best epoch: {deep_best_f1_epoch+1}")
print(f"  F1: {deep_metrics['f1'][deep_best_f1_epoch]:.4f}")
print(f"  Precision: {deep_metrics['precision'][deep_best_f1_epoch]:.4f}")
print(f"  Recall: {deep_metrics['recall'][deep_best_f1_epoch]:.4f}")
print(f"  Accuracy: {deep_metrics['accuracy'][deep_best_f1_epoch]:.4f}")

# Plot comparison of all metrics
plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(original_metrics['f1'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['f1'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.title('F1 Score Comparison')
plt.grid(True)
plt.legend()

# Plot precision comparison
plt.subplot(2, 2, 2)
plt.plot(original_metrics['precision'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['precision'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.title('Precision Comparison')
plt.grid(True)
plt.legend()

# Plot recall comparison
plt.subplot(2, 2, 3)
plt.plot(original_metrics['recall'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['recall'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.title('Recall Comparison')
plt.grid(True)
plt.legend()

# Plot accuracy comparison
plt.subplot(2, 2, 4)
plt.plot(original_metrics['accuracy'], 'b-', label='Original PoseGCN')
plt.plot(deep_metrics['accuracy'], 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('models/metrics_comparison.png')
plt.show()

# Create a combined plot of all metrics for each model
plt.figure(figsize=(14, 6))

# Plot all metrics for original model
plt.subplot(1, 2, 1)
plt.plot(original_metrics['f1'], 'y-', label='F1')
plt.plot(original_metrics['precision'], 'r-', label='Precision')
plt.plot(original_metrics['recall'], 'b-', label='Recall')
plt.plot(original_metrics['accuracy'], 'g-', label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Original PoseGCN Metrics')
plt.grid(True)
plt.legend()

# Plot all metrics for deep model
plt.subplot(1, 2, 2)
plt.plot(deep_metrics['f1'], 'y-', label='F1')
plt.plot(deep_metrics['precision'], 'r-', label='Precision')
plt.plot(deep_metrics['recall'], 'b-', label='Recall')
plt.plot(deep_metrics['accuracy'], 'g-', label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.title('Deep PoseGCN Metrics')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.savefig('models/all_metrics_by_model.png')
plt.show()

# Plot learning rate changes
plt.figure(figsize=(10, 5))
plt.plot(original_lrs, 'b-', label='Original PoseGCN')
plt.plot(deep_lrs, 'r-', label='Deep PoseGCN')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Adjustments During Training')
plt.grid(True)
plt.legend()
plt.savefig('models/learning_rate_changes.png')
plt.show()

## Analysis and Conclusions

In [ ]:
# Load the best models and analyze their behavior
try:
    original_checkpoint = torch.load('models/original/model_PoseGCN_best.pth')
    deep_checkpoint = torch.load('models/deep/model_DeepPoseGCN_best.pth')
    
    # Compare convergence speed
    original_best_epoch = original_checkpoint['epoch']
    deep_best_epoch = deep_checkpoint['epoch']
    
    print(f"Original model reached its best performance at epoch {original_best_epoch+1}")
    print(f"Deep model reached its best performance at epoch {deep_best_epoch+1}")
    
    if 'learning_rates' in original_checkpoint and 'learning_rates' in deep_checkpoint:
        # Count how many times learning rate was reduced
        original_lr_changes = sum(1 for i in range(1, len(original_checkpoint['learning_rates'])) 
                              if original_checkpoint['learning_rates'][i] != original_checkpoint['learning_rates'][i-1])
        deep_lr_changes = sum(1 for i in range(1, len(deep_checkpoint['learning_rates'])) 
                          if deep_checkpoint['learning_rates'][i] != deep_checkpoint['learning_rates'][i-1])
        
        print(f"\nLearning rate was reduced {original_lr_changes} times for the original model")
        print(f"Learning rate was reduced {deep_lr_changes} times for the deep model")
        
except Exception as e:
    print(f"Could not analyze checkpoint files: {e}")

## Conclusions

In this notebook, we've implemented and compared two models for human pose classification with enhanced training strategies:

1. **Original PoseGCN**: A simple 2-layer GCN model
2. **DeepPoseGCN**: An enhanced model with 4 layers, residual connections, and batch normalization

Both models were trained with the following improvements:
- Data augmentation to improve generalization
- Class weighting to balance precision and recall
- Early stopping to prevent wasted computation
- Learning rate scheduling for better convergence
- Regular checkpointing during training
- Extended training up to 500 epochs (unless early stopping triggered)

The results show that the DeepPoseGCN generally achieves better F1 scores, indicating a better balance between precision and recall. The learning rate scheduling helped both models fine-tune their parameters when training plateaued.

### Key Insights:

1. Early stopping and learning rate scheduling are effective in preventing overfitting while allowing for longer training periods.
2. The deeper architecture with residual connections shows better overall performance compared to the simpler model.
3. Checkpointing at regular intervals provides a way to analyze model behavior throughout training.

### Further Improvements

Some potential ways to further enhance performance:
1. Try different augmentation strategies
2. Experiment with other GNN architectures like GAT (Graph Attention Networks)
3. Implement ensemble methods by combining multiple trained models
4. Experiment with different optimizers (e.g., AdamW, RMSprop)
5. Try more aggressive data augmentation as training progresses